In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [2]:
sys.path.append(r"../../src/collectcube")
from db import *
from db_post import *

In [3]:
out_dir = "C:/GISprojects/ParaguayTraining"
local_db_path = os.path.join(out_dir, 'py_training.db')

In [4]:
table_check = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
277510  0031482_4       0  3081000.0 -3094060.0 -24.458177 -56.400685   
277511  0031482_5       0  3081020.0 -3094060.0 -24.458177 -56.400467   
277512  0031482_6       0  3081000.0 -3094070.0 -24.458259 -56.400675   
277513  0031482_7       0  3081010.0 -3094070.0 -24.458259 -56.400566   
277514  0031482_8       0  3081020.0 -3094070.0 -24.458259 -56.400457   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2 biome  \
277510        1        0  31482     4  GE_extra  0.564274  0.539983    BH   
277511        1        0  31482     5  GE_extra  0.564274  0.539983    BH   
277512        1        0  31482     6  GE_extra  0.564274  0.539983    BH   
277513        1        0  31482     7  GE_extra  0.564274  0.539983    BH   
277514        1        0  31482     8  GE_extra  0.564274  0.539983    BH   

        gridCell  
277510      3867  
277511      3867  
277512      3867  
277513      3867  
277

## Open db table as pandas dataframe

## Get sample for target date (direct and indirect)

In [5]:
## Will consider anything 1-June of prior year to 1-June of target_date year as direct observation 
## and interpolate from other observations according to the rules set up and the 'stability' column in LUT
## if sample_type is 'area_est' or 'validation', will only include center pixels

target_yr = 2022

sample_type = 'training' ## 'area_est',  validation', 'training' 
sample_cats = 'LC25'   ## 'bi_tree', 'bi_forest'
lut = '../../data/Class_LUT.csv'
   
all_obs=get_sample_for_date(target_yr,local_db_path,sample_type,sample_cats,lut)
    
all_obs.sort_values('PID0')
all_obs.tail(n=10)

0 is not in LUT
there are 27856 observations for selected date


,recID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,PercentTree,...,cent_lat,cent_long,ransamp,checked,sampgroup,rand,rand2,biome,gridCell,LC2
27846,93066.0,31454,0,2023-06-06,20,26,3.0,None,None,None,...,-24.090690,-56.546396,0,0,GE_extra,0.692447,0.825737,BH,3829,0
27847,93071.0,31454,1,2023-06-06,20,26,5.0,None,None,None,...,-24.090609,-56.546515,0,0,GE_extra,0.692447,0.825737,BH,3829,0
27848,93075.0,31455,0,2023-06-06,70,66,8.0,None,None,None,...,-24.102268,-56.543379,0,0,GE_extra,0.988693,0.472295,BH,3829,0
27849,93143.0,31466,0,2022-01-01,70,54,8.0,None,None,None,...,-24.694270,-56.040257,0,0,GE_extra,0.542753,0.451004,BH,3904,0
27850,93150.0,31467,0,2022-01-01,70,54,7.0,None,None,None,...,-24.697948,-56.042310,0,0,GE_extra,0.753376,0.282749,BH,3904,0
27851,93157.0,31468,0,2023-03-30,40,46,8.0,None,None,None,...,-24.701217,-56.044631,0,0,GE_extra,0.853632,0.675750,BH,3904,0
27852,93161.0,31469,0,2023-10-01,70,66,8.0,None,None,None,...,-24.691327,-55.999912,0,0,GE_extra,0.721903,0.128903,BH,3904,0
27853,93170.0,31472,0,2021-06-12,40,41,8.0,None,None,None,...,-27.239530,-55.813895,1,0,GE_extra,0.907977,0.050668,BH,3919,0
27854,93177.0,31473,0,2021-06-12,20,31,8.0,None,None,None,...,-27.239860,-55.811204,1,0,GE_extra,0.972399,0.391651,BH,3919,0
27855,93182.0,31474,0,2021-06-12,40,41,8.0,None,None,None,...,-27.240026,-55.813605,1,0,GE_extra,0.356496,0.106146,BH,3919,0


In [8]:
#all_obs.to_csv(os.path.join(out_dir,'2022obscheck.csv'))

### Separate fixed holdouts from 2021 sample

In [29]:
### Separates all pixels with same PID0 into houldout to avoid training on one pix and testing on another from same group
###    this means that HO sets should probably be filtered to reduce redundancy later

lut = '../../data/Class_LUT.csv'

## first run above for target year 2021
cutoff = .20
mixedcrop_lcs = [23,24,25,26,32,34,35,36,39]
big_crops = [22,30,31,33,37,38]
med_crops = [40,41,42,43,45,46,47,54]

ho_set = all_obs[all_obs['rand2']<=cutoff]

fixed_ho_sm = ho_set[ho_set['LC'].isin(mixedcrop_lcs)]
print(f'there are {len(fixed_ho_sm)} pixels in the small crop holdout \n')
fixed_ho_sm.to_csv(os.path.join(out_dir,'GENERAL_HOLDOUT_smallCrop.csv'))

ho_obs2 = ho_set[ho_set['LC'].isin(big_crops)]
fixed_ho_big = ho_obs2[~(ho_obs2['Width']<= 120)]
print(f'there are {len(fixed_ho_big)} pixels in the big crop holdout \n')
fixed_ho_big.to_csv(os.path.join(out_dir,'GENERAL_HOLDOUT_bigCrop.csv'))

fixed_ho_med = ho_set[ho_set['LC'].isin(med_crops)]
print(f'there are {len(fixed_ho_med)} pixels in the med crop holdout \n')
fixed_ho_med.to_csv(os.path.join(out_dir,'GENERAL_HOLDOUT_medCrop.csv'))

ho_obs4 = ho_set[(~ho_set['LC'].isin(mixedcrop_lcs)) & (~ho_set['LC'].isin(big_crops)) & (~ho_set['LC'].isin(med_crops))]
fixed_ho_nocrop = balance_training_data(target_yr, lut, ho_obs4, out_dir, 1, 1)
print(f'there are {len(fixed_ho_nocrop)} pixels in the no crop holdout \n')
fixed_ho_nocrop.to_csv(os.path.join(out_dir,'GENERAL_HOLDOUT_noCrop.csv'))
## note Mixed-FieldEdge is still in here as noncrop. Probably want to remove later

all_hos = pd.concat([fixed_ho_sm,fixed_ho_big,fixed_ho_med,fixed_ho_nocrop])
non_hos1 = all_obs[~all_obs['PIDi'].isin(all_hos['PIDi'])]
non_hos0 = all_obs[all_obs['rand2']>cutoff]  ## this is safer to use for training data
print(f'there are {len(non_hos1)} pixels remaining, but training set will use {len(non_hos0)} \n')
#non_hos0.to_csv(os.path.join(out_dir,'GENERAL_TRAINING.csv'))
tot_ho = len(fixed_ho_sm)+len(fixed_ho_big)+len(fixed_ho_med)+len(fixed_ho_nocrop)
print(f'check: there should be {len(all_obs)} - {tot_ho} = {len(all_obs) - tot_ho}')

there are 624 pixels in the small crop holdout 

there are 996 pixels in the big crop holdout 

there are 289 pixels in the med crop holdout 

             LC25_name  counts
0           Mixed-path    2323
1      Mixed-FieldEdge    1205
2         Trees-Forest     428
3    Grassland-Managed     421
4      Trees-disturbed     418
5          NoVeg_Built     364
6    Grassland-Natural     278
7      Mixed-GrassEdge     277
8              unknown     268
9        Grassland-Wet     266
10          NoVeg_Bare     249
11               Shrub     197
12           TreePlant     180
13      Grass_tree-mix     100
14         NoVeg_Water      97
15  Trees-Forest_shrub      61
16   Trees-Forest_palm      43
17           Trees-wet      27
18             Cleared      23
19         Burnt-woody      18
20            NewPlant      10
Total sample size before balancing is: 7253
tot percent =: 1.00923
samp size for class with max proportion is 428.0
    perLC25E           LC25_name  counts  p1_counts  p2_cou

### remove the above holdout from all dfs prior to balancing?

## Balance sample by class and quality

In [68]:
lut=Path('../../data/Class_LUT.csv')
## note non_hos0 is just all available pixels with rand2 > .2
#print(pd.read_csv(lut).sort_values('LC_UNQ')[['LC_UNQ','USE_NAME','LC25','LC25_name','LC3_name']])
cutoff = 100
mixbal = 10
dfbal = balance_training_data(target_yr, lut, non_hos0, out_dir, cutoff, mixbal)

             LC25_name  counts
0   Crops-SoyCornWheat    3584
1            Crops-mix    2133
2    Grassland-Managed    1687
3         Trees-Forest    1648
4              unknown    1226
5        Grassland-Wet    1027
6    Grassland-Natural     952
7      Trees-disturbed     742
8            TreePlant     623
9     Crops-Yerba-Mate     498
10               Shrub     490
11          Mixed-path     440
12         NoVeg_Water     403
13      Grass_tree-mix     367
14     Mixed-FieldEdge     254
15         NoVeg_Built     248
16     Mixed-GrassEdge     235
17  Trees-Forest_shrub     226
18        Crops-Banana     216
19   Trees-Forest_palm     215
20          NoVeg_Bare     202
21         Crops-Sugar     177
22       Crops-Orchard     172
23             Cleared     134
24          Crops-Rice     121
25           Trees-wet      79
26      Crops-Mandioca      74
27        Crops-Sesame      63
28         Burnt-woody      56
29        Crops-Cotton      53
30       Crops-Shrub47      42
31  Crop

## Summarize sample size by sample group